In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to c:\users\sec\appdata\local\temp\pip-install-da7eqfd4\gym-walk_020a6f549f7841f49f6dc878f20eeea4
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk 'C:\Users\SEC\AppData\Local\Temp\pip-install-da7eqfd4\gym-walk_020a6f549f7841f49f6dc878f20eeea4'


In [36]:

import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [37]:
from itertools import count
from tqdm import tqdm

In [38]:

def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [39]:

def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [40]:
def probability_success(env, pi, goal_state=15, n_episodes=100, max_steps=200):
    np.random.seed(123)
    results = []

    for _ in range(n_episodes):
        state, _ = env.reset(seed=123)  # Correct seeding method
        for _ in range(max_steps):
            action = pi[state]
            state, _, terminated, truncated, _ = env.step(action)
            if state == goal_state:
                results.append(1)
                break
            if terminated or truncated:
                results.append(0)
                break
    return np.mean(results)


In [41]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    np.random.seed(123)
    returns = []

    for _ in range(n_episodes):
        state, _ = env.reset(seed=123)
        total_reward = 0
        for _ in range(max_steps):
            action = pi[state]
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            if terminated or truncated:
                break
        returns.append(total_reward)
    return np.mean(returns)


In [42]:

env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
#LEFT, RIGHT = range(2)

In [43]:
P


{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [44]:
def decay_schedule(init_value,min_value, decay_ratio,max_steps,log_start=-2,log_base=10):

  decay_steps = int(max_steps*decay_ratio)
  rem_steps = max_steps - decay_steps
  values = np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
  values=(values-values.min()) / (values.max()-values.min())

  values=(init_value - min_value)*values + min_value
  values = np.pad(values,(0,rem_steps),'edge')
  return values


In [45]:
def generate_trajectory(select_action, Q, epsilon, env, max_steps):
    trajectory = []
    state, _ = env.reset()
    for _ in range(max_steps):
        action = select_action(state, Q, epsilon)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        trajectory.append((state, action, reward, next_state, done))
        if done:
            break
        state = next_state
    return trajectory

In [46]:
def mc_control(env, gamma=0.99,
               init_alpha=1.0, min_alpha=0.02, alpha_decay_ratio=0.5,
               init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
               n_episodes=9000, max_steps=200, first_visit=True):

    nS = env.observation_space.n
    nA = env.action_space.n

    Q = np.zeros((nS, nA))
    Q_track = np.zeros((n_episodes, nS, nA))
    pi = np.zeros(nS, dtype=int)

    def select_action(state, Q, epsilon):
        if np.random.rand() > epsilon:
            return np.argmax(Q[state])
        else:
            return np.random.randint(nA)

    alphas = np.linspace(init_alpha, min_alpha, int(n_episodes * alpha_decay_ratio))
    alphas = np.concatenate([alphas, np.full(n_episodes - len(alphas), min_alpha)])

    epsilons = np.linspace(init_epsilon, min_epsilon, int(n_episodes * epsilon_decay_ratio))
    epsilons = np.concatenate([epsilons, np.full(n_episodes - len(epsilons), min_epsilon)])

    for e in tqdm(range(n_episodes), leave=False):
        trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)
        visited = np.zeros((nS, nA), dtype=bool)
        for t, (state, action, reward, _, _) in enumerate(trajectory):
            if first_visit and visited[state][action]:
                continue
            visited[state][action] = True
            n_steps = len(trajectory[t:])
            discounts = np.array([gamma ** i for i in range(n_steps)])
            rewards = [step[2] for step in trajectory[t:]]
            G = np.sum(discounts * rewards)
            Q[state][action] += alphas[e] * (G - Q[state][action])
        Q_track[e] = Q

    V = np.max(Q, axis=1)
    pi = np.argmax(Q, axis=1)

    return Q, V, pi

In [47]:
def print_policy(pi, P, action_symbols=['<', 'v', '>', '^'], n_cols=4, title='Policy:'):
    print(title)
    arrs = {k: v for k, v in enumerate(action_symbols)}
    for s in range(len(P)):
        print("| ", end="")
        if np.all([done for action in P[s].values() for (_, _, _, done) in action]):
            print("".rjust(9), end=" ")
        else:
            a = pi[s]  # ✅ Use indexing, not function call
            print(arrs[a].rjust(9), end=" ")
        if (s + 1) % n_cols == 0:
            print("|")


In [48]:

optimal_Q, optimal_V, optimal_pi = mc_control (env,n_episodes = 9000)
print('SHALINI K\n 212222240095\n')
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)

SHALINI K
 212222240095

Action-value function:
| 00 [0.28 0.23 0.28 0.24] | 01 [0.14 0.15 0.11 0.25] | 02 [0.25 0.15 0.15 0.14] | 03 [0.06 0.05 0.03 0.15] |
| 04 [0.29 0.24 0.2  0.19] |           | 06 [0.21 0.08 0.12 0.05] |           |
| 08 [0.26 0.29 0.29 0.32] | 09 [0.29 0.42 0.31 0.24] | 10 [0.42 0.35 0.33 0.19] |           |
|           | 13 [0.27 0.46 0.59 0.37] | 14 [0.45 0.76 0.68 0.56] |           |
State-value function:
| 00   0.28 | 01   0.25 | 02   0.25 | 03   0.15 |
| 04   0.29 |           | 06   0.21 |           |
| 08   0.32 | 09   0.42 | 10   0.42 |           |
|           | 13   0.59 | 14   0.76 |           |
Policy:
|         < |         ^ |         < |         ^ |
|         < |           |         < |           |
|         ^ |         v |         < |           |
|           |         > |         v |           |


In [49]:
print('SHALINI K\n212222240095\n')
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=15) * 100,
    mean_return(env, optimal_pi)))


SHALINI K
212222240095

Reaches goal 100.00%. Obtains an average undiscounted return of 1.0000.
